In [1]:

import numpy as np
from scipy.ndimage import gaussian_filter
import os
from soma import aims

from sulci.registration.spam import spam_register

import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma.aimsalgo import MorphoGreyLevel_S16

# Global static variables
_AIMS_BINARY_ONE = 32767
_dilation = 1
_threshold = 0

# launching anatomist
a = ana.Anatomist()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [2]:
def dilate(mask, radius=_dilation):
    """Makes a dilation radius _dlation, in mm
    """
    arr = mask.np
    # Binarization of mask
    arr[arr < 1] = 0
    arr[arr >= 1] = _AIMS_BINARY_ONE
    # Dilates initial volume of 10 mm
    morpho = MorphoGreyLevel_S16()
    dilate = morpho.doDilation(mask, radius)
    arr_dilate = dilate.np
    arr_dilate[arr_dilate >= 1] = 1
    return dilate

In [3]:
spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/Sc.Cal.-S.Li._left.nii.gz'
skel_f = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_ammon.nii.gz' 

# Reads spam and skeleton files
mask_result = aims.read(spam_file)
skel_data = aims.read(skel_f, dtype="Volume_FLOAT")
skel_data.np[:] = (skel_data.np > 0).astype(np.float32)

# Makes binarization and dilation on spam
mask_result[mask_result.np <= _threshold] = 0
mask_result.np[:] = dilate(mask_result).np

DILATION


In [4]:
# Masks skeleton data with dilated spam
skel_data.np[mask_result.np <= 0] = 0
skel_data.np[:] = np.roll(skel_data.np, 5, axis=0)
skel_data.np[:] = 100*gaussian_filter(skel_data.np, sigma=2)
aims.write(skel_data, "/tmp/skel_before.nii.gz")

In [5]:
# Reads initial spam volume and transforms it to proba
spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")
spam_vol.np[:] = spam_vol.np

aims.write(spam_vol, "/tmp/spam.nii.gz")

In [6]:
np.unique(spam_vol.np)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34.], dtype=float32)

In [7]:
np.unique(skel_data, return_counts=True)

(array([0.0000000e+00, 2.9963709e-11, 1.9538791e-10, ..., 2.1985331e+01,
        2.2006870e+01, 2.2116522e+01], dtype=float32),
 array([1003817,     172,     116, ...,       1,       1,       1]))

In [8]:
# Makes realignmet
os.system(f"AimsMIRegister -r /tmp/spam.nii.gz  -t /tmp/skel_before.nii.gz --dir /tmp/transform.trm")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


objective function   : mutual information
optimizer            : powell
Reading channel -1 of reference image
Reference Image       : /tmp/spam.nii.gz
         Pyramid range: 0 - 0
Test      Image       : /tmp/skel_before.nii.gz
         Pyramid range: 0 - 0
Number of Level       : 64
Parzen Mask Size      : 5
Error (Stop Criterion): 0.01
Ref Threshold  ratio  : 0.05
Test Threshold ratio  : 0.1
initial parameter     : (0, 0, 0, 0, 0, 0)
parameter variations  : (0, 0, 0, 0, 0, 0)
Processing Plevel 0(0, 0, 0, 0, 0, 0)(1, 1, 0.596832, 1, 0.596832, 0.502595)
19817091 ticks
Niveau : 0 Popt (1.45539, -0.400064, -0.000292318, -0.0489621, -0.00345183, 0.000861318)
Volume 0Popt (1.45539, -0.400064, -0.000292318, -0.0489621, -0.00345183, 0.000861318)
 -Tx 1.45539 -Ty -0.400064 -Tz -0.0489621 -Rx -0.00345183 -Ry 0.000861318 -Rz -0.000292318
Direct FileName /tmp/transform.trm
Inverse FileName /tmp/spam.nii_TO_skel_before.nii.trm


0

In [9]:
# Applies the realignment
os.system(f"AimsApplyTransform -i /tmp/skel_before.nii.gz -o /tmp/skel_realigned.nii.gz -m /tmp/transform.trm -I /tmp/spam.nii_TO_skel_before.nii.trm")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loading direct transformations
loading inverse transformations
Output dimensions: 96, 114, 96
Output voxel size: 2, 2, 2 mm
Resampling carto_volume of FLOAT...   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 %


0

In [10]:
spam = a.loadObject(spam_file)
skel = a.loadObject("/tmp/skel_before.nii.gz")
realigned = a.loadObject("/tmp/skel_realigned.nii.gz")
w = a.createWindow('Sagittal')
skel.setPalette("Blues")
realigned.setPalette("Reds")
w.addObjects(spam)
w.addObjects(skel)
w.addObjects(realigned)

Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


observable 0x614c48226030(N9anatomist7AVolumeIfEE) could not be removed from observer 0x614c47fd8fd8 (N9anatomist8Fusion2DE)
